In [1]:
%matplotlib inline
import os
import numpy as np
import scipy
import matplotlib.pyplot as plt

import vdmlab as vdm
from core import Rat, vdm_assign_label, combine_rats
from load_data import get_events
from plotting import plot_behavior

In [2]:
output_filepath = 'C:/Users/Emily/code/emi_biconditional/plots'

In [3]:
import info.RH01d1 as RH01d1

In [4]:
def correct_timestamps(events):
    events['cue_on'] = events['cue']
    
    events['cue_off'] = []
    for on in events['cue_on']:
        for off in events['main_off']:
            if np.allclose(on+10, off, atol=0.5):
                events['cue_off'].append(off)
    events['cue_off'] = np.array(events['cue_off'])
    
    events['house_on'] = [events['house'][0], events['house'][7], events['house'][14], events['house'][21]]
    events['house_on'] = np.array(events['house_on'])
    
    events['flash_off'] = [events['house'][6], events['house'][13], events['house'][20], events['house'][-1]]
    
    events['house_off'] = []
    for last in events['flash_off']:
        for off in events['main_off']:
            if np.allclose(last, off, atol=0.5):
                events['house_off'].append(off)
    events['house_off'] = np.array(events['house_off'])
    
    return events

In [5]:
this_rat = '1'

rats = [this_rat]
data = dict()
data[this_rat] = Rat(this_rat)

sessions = [RH01d1]

for session in sessions:
    events = get_events(session.event_mat)
    events = correct_timestamps(events)
    
    rats_data = vdm_assign_label(events)

    data[this_rat].add_session(**rats_data, n_unique=2, delay=5.0, tolerance=1.7)
    
n_sessions = len(data[this_rat].sessions)

df = combine_rats(data, rats, n_sessions)

In [6]:
df.head()

,condition,cue,measure,rat,rewarded,session,trial,trial_type,value
0,light 1,light,durations,1,light unrewarded,1,"1, 0",1,0.0000
1,light 1,light,numbers,1,light unrewarded,1,"1, 0",1,0.0000
2,light 1,light,latency,1,light unrewarded,1,"1, 0",1,10.0000
3,light 1,light,responses,1,light unrewarded,1,"1, 0",1,0.0000
4,sound 1,sound,durations,1,sound unrewarded,1,"1, 1",1,1.7681


In [ ]:
filename = 'vdmlab_trials_rat1_behavior.png'
filepath = os.path.join(output_filepath, filename)
plot_behavior(df, ['1'], filepath, only_sound=False, by_outcome=False)

In [ ]:
events = get_events(RH01d1.event_mat)
events['label']

In [ ]:
events.keys()

In [ ]:
events['cue']

In [ ]:
# plt.plot(events['start_recording'], np.ones(len(events['start_recording'])),'|', color='k', ms=45)
# plt.plot(events['stop_recording'], np.ones(len(events['stop_recording'])),'|', color='k', ms=45)
# plt.plot(events['main_off'], np.ones(len(events['main_off'])),'|', color='g', ms=50)
# plt.plot(events['cue'], np.ones(len(events['cue'])),'|', color='c', ms=35)
plt.plot(events['house'], np.ones(len(events['house'])),'|', color='m', ms=35)
# plt.plot(events['feeder'], np.ones(len(events['feeder'])),'|', color='r', ms=15)
# plt.plot(events['noise_on'], np.ones(len(events['noise_on'])),'|', color='b', ms=25)
# plt.plot(events['tone_on'], np.ones(len(events['tone_on'])),'|', color='g', ms=25)
# plt.plot(events['noise_off'], np.ones(len(events['noise_off'])),'|', color='y', ms=25)
# plt.plot(events['tone_off'], np.ones(len(events['tone_off'])),'|', color='k', ms=25)

# plt.plot(events['pb_on'], np.ones(len(events['pb_on'])),'|', color='#c994c7', ms=35)
# plt.plot(events['pb_off'], np.ones(len(events['pb_off'])),'|', color='#636363', ms=25)
plt.ylim(0.95, 1.05)
plt.show()

In [ ]:
len(events['pb_on']), len(events['pb_off'])

In [ ]:
events['tone_off'][0] - events['tone_on'][0]

In [ ]:
events['cue_on'] = events['cue']

In [ ]:
events['main_off']

In [ ]:
events['cue_off'] = []
for on in events['cue']:
    for off in events['main_off']:
        if np.allclose(on+10, off, atol=0.5):
            events['cue_off'].append(off)

In [ ]:
events['cue_off'] = np.array(events['cue_off'])
events['cue_off']

In [ ]:
events['house_on'] = [events['house'][0], events['house'][7], events['house'][14], events['house'][21]]
events['house_on'] = np.array(events['house_on'])

In [ ]:
events['flash_off'] = [events['house'][6], events['house'][13], events['house'][20], events['house'][-1]]

In [ ]:
events['house_off'] = []
for last in events['flash_off']:
    for off in events['main_off']:
        if np.allclose(last, off, atol=0.5):
            events['house_off'].append(off)
events['house_off'] = np.array(events['house_off'])

In [ ]:
i = 0
events['house_off'][i] - events['house_on'][i]

In [ ]:
events['tone_off']

In [ ]:
events['tone_on']

In [ ]:
events['noise_on']

In [ ]:
events['noise_off']

In [ ]:
events['house_on']

In [ ]:
events['house_off']

In [ ]:
events['cue_on']

In [ ]:
events['cue_off']

In [ ]:
len(events['pb_on']), len(events['pb_off'])

In [ ]:
events['feeder']

In [ ]:
rats_data = vdm_assign_label(events)

In [ ]:
for session in sessions:
    events = get_events(session.event_mat)